In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.pyplot as plt
import librosa
import librosa.display
import IPython.display as ipd
from scipy.io import wavfile as wav
import pandas as pd
import os
import numpy as np
import seaborn as sns
import scipy

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, XGBRFClassifier

from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [ ]:
!pip install pydub

In [ ]:
from pydub import AudioSegment
from pydub.utils import make_chunks

In [ ]:
# import required module
import os
# assign directory
directory = '/content/drive/MyDrive/Data/genres_original'

file_dict = {}
# iterate over files in
# that directory
for foldername in os.listdir(directory):
    folder = os.path.join(directory, foldername)
    files = []
    for filename in os.listdir(folder):
      f = os.path.join(folder, filename)
      files.append(f)
    file_dict[foldername] = files

In [ ]:
cnt = 0
bt = 0
pre = '/content/drive/MyDrive/Data/genres_original'
for folder,file_list in file_dict.items():
  for f in file_list:
    x = f.replace(pre+'/'+str(folder)+'/','')
    y = os.path.splitext(x)
    name = y[0]
    try:
      myaudio = AudioSegment.from_file(f , "wav") 
      chunk_length_ms = 3000 # pydub calculates in millisec
      chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of three sec
      #Export all of the individual chunks as wav files

      for i, chunk in enumerate(chunks):
          chunk_name = f'{name}_chunk_{i}.wav'
          if not os.path.exists('/content/drive/MyDrive/3sec_audio_files/'+folder):
            os.makedirs('/content/drive/MyDrive/3sec_audio_files/'+folder)
            chunk.export('/content/drive/MyDrive/3sec_audio_files/'+folder+'/'+chunk_name, format="wav")
          else:
            chunk.export('/content/drive/MyDrive/3sec_audio_files/'+folder+'/'+chunk_name, format="wav")
    except:
      bt = bt+1
      pass
    print(cnt,bt,folder)
    cnt = cnt+1


In [ ]:

# import required module
import os
# assign directory
directory = '/content/drive/MyDrive/3sec_audio_files'

file_dict = {}
# iterate over files in
# that directory
for foldername in os.listdir(directory):
    folder = os.path.join(directory, foldername)
    files = []
    for filename in os.listdir(folder):
      f = os.path.join(folder, filename)
      files.append(f)
    file_dict[foldername] = files

In [ ]:
lst = ['chroma_mean_0', 'chroma_mean_1','chroma_mean_2','chroma_mean_3','chroma_mean_4','chroma_mean_5','chroma_mean_6','chroma_mean_7','chroma_mean_8',
'chroma_mean_9','chroma_mean_10','chroma_mean_11','chroma_std_0','chroma_std_1','chroma_std_2','chroma_std_3','chroma_std_4','chroma_std_5','chroma_std_6',
'chroma_std_7','chroma_std_8','chroma_std_9','chroma_std_10','chroma_std_11','mfccs_mean_0','mfccs_mean_1','mfccs_mean_2','mfccs_mean_3','mfccs_mean_4',
 'mfccs_mean_5','mfccs_mean_6','mfccs_mean_7','mfccs_mean_8','mfccs_mean_9','mfccs_mean_10','mfccs_mean_11','mfccs_mean_12','mfccs_std_0','mfccs_std_1',
 'mfccs_std_2','mfccs_std_3','mfccs_std_4','mfccs_std_5','mfccs_std_6','mfccs_std_7','mfccs_std_8','mfccs_std_9','mfccs_std_10','mfccs_std_11','mfccs_std_12',
 'cent_mean','cent_std','cent_skew','contrast_mean_0','contrast_mean_1','contrast_mean_2','contrast_mean_3','contrast_mean_4','contrast_mean_5','contrast_mean_6',
 'contrast_std_0','contrast_std_1','contrast_std_2','contrast_std_3','contrast_std_4','contrast_std_5','contrast_std_6','rolloff_mean','rolloff_std','rolloff_skew',
 'zrate_mean','zrate_std','zrate_skew','tempo','label']

In [ ]:
def chroma_feature_extraction(y_harmonic,sr,idx):
  chroma=librosa.feature.chroma_cens(y=y_harmonic, sr=sr)
  chroma_mean=np.mean(chroma,axis=1)
  chroma_std=np.std(chroma,axis=1)
  octave=['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']
  chroma_df=pd.DataFrame()
  for i in range(0,12):
      chroma_df['chroma_mean_'+str(i)]=chroma_mean[i]
  for i in range(0,12):
      chroma_df['chroma_std_'+str(i)]=chroma_mean[i]
  chroma_df.loc[idx]=np.concatenate((chroma_mean,chroma_std),axis=0)

  return chroma_df

def mfcc_feature_extraction(y_harmonic,sr,idx):
  mfccs = librosa.feature.mfcc(y=y_harmonic, sr=sr, n_mfcc=13)
  mfccs_mean=np.mean(mfccs,axis=1)
  mfccs_std=np.std(mfccs,axis=1)

  coeffs=np.arange(0,13)
  mfccs_df=pd.DataFrame()
  for i in range(0,13):
      mfccs_df['mfccs_mean_'+str(i)]=mfccs_mean[i]
  for i in range(0,13):
      mfccs_df['mfccs_std_'+str(i)]=mfccs_mean[i]
  mfccs_df.loc[idx]=np.concatenate((mfccs_mean,mfccs_std),axis=0)

  return mfccs_df

def cent_feature_extraction(y,y_harmonic,sr,idx):
  import scipy
  cent = librosa.feature.spectral_centroid(y=y, sr=sr)
  contrast=librosa.feature.spectral_contrast(y=y_harmonic,sr=sr)
  rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
  cent_mean=np.mean(cent)
  cent_std=np.std(cent)
  cent_skew=scipy.stats.skew(cent,axis=1)[0]
  contrast_mean=np.mean(contrast,axis=1)
  contrast_std=np.std(contrast,axis=1)
  rolloff_mean=np.mean(rolloff)
  rolloff_std=np.std(rolloff)
  rolloff_skew=scipy.stats.skew(rolloff,axis=1)[0]
  spectral_df=pd.DataFrame()
  collist=['cent_mean','cent_std','cent_skew']
  for i in range(0,7):
      collist.append('contrast_mean_'+str(i))
  for i in range(0,7):
      collist.append('contrast_std_'+str(i))
  collist=collist+['rolloff_mean','rolloff_std','rolloff_skew']
  for c in collist:
      spectral_df[c]=0
  data=np.concatenate(([cent_mean,cent_std,cent_skew],contrast_mean,contrast_std,[rolloff_mean,rolloff_std,rolloff_std]),axis=0)
  spectral_df.loc[idx]=data

  return spectral_df

def zcrossrate_feature_extraction(y_harmonic,idx):
  zrate=librosa.feature.zero_crossing_rate(y_harmonic)
  zrate_mean=np.mean(zrate)
  zrate_std=np.std(zrate)
  zrate_skew=scipy.stats.skew(zrate,axis=1)[0]
  zrate_df=pd.DataFrame()
  zrate_df['zrate_mean']=0
  zrate_df['zrate_std']=0
  zrate_df['zrate_skew']=0
  zrate_df.loc[idx]=[zrate_mean,zrate_std,zrate_skew]

  return zrate_df

def beat_tempo_feature_extraction(y_harmonic,sr,idx):
  tempo, beat_frames = librosa.beat.beat_track(y=y_harmonic, sr=sr)
  beat_df=pd.DataFrame()
  beat_df['tempo']=tempo
  beat_df.loc[idx]=tempo
  
  return beat_df

def label_extractor(folder,idx):
  label_df=pd.DataFrame()
  label_df['label']= folder
  label_df.loc[idx]= folder
  
  return label_df


In [ ]:
idx = 0
newly_engneered_df = pd.DataFrame(columns = lst)
for folder,file_list in file_dict.items():
  for f in file_list:
    try:
      y, sr = librosa.load(f)
      y_harmonic, y_percussive = librosa.effects.hpss(y)
      chroma_df=chroma_feature_extraction(y_harmonic,sr,idx)
      mfccs_df=mfcc_feature_extraction(y_harmonic,sr,idx)
      spectral_df=cent_feature_extraction(y,y_harmonic,sr,idx)
      zrate_df=zcrossrate_feature_extraction(y_harmonic,idx)
      beat_df=beat_tempo_feature_extraction(y_harmonic,sr,idx)
      label_df = label_extractor(folder,idx)
      temp=pd.concat((chroma_df,mfccs_df,spectral_df,zrate_df,beat_df,label_df),axis=1)
      print(temp.shape)
      newly_engneered_df.loc[idx] = temp.loc[idx]
      idx = idx+1
      print(idx,f)
    except:
      pass
newly_engneered_df.to_csv('/content/drive/MyDrive/Data/new_features_3_sec.csv')

(1, 75)
1 /content/drive/MyDrive/3sec_audio_files/blues/blues.00000_chunk_0.wav
(1, 75)
2 /content/drive/MyDrive/3sec_audio_files/blues/blues.00000_chunk_1.wav
(1, 75)
3 /content/drive/MyDrive/3sec_audio_files/blues/blues.00000_chunk_2.wav
(1, 75)
4 /content/drive/MyDrive/3sec_audio_files/blues/blues.00000_chunk_3.wav
(1, 75)
5 /content/drive/MyDrive/3sec_audio_files/blues/blues.00000_chunk_4.wav
(1, 75)
6 /content/drive/MyDrive/3sec_audio_files/blues/blues.00000_chunk_5.wav
(1, 75)
7 /content/drive/MyDrive/3sec_audio_files/blues/blues.00000_chunk_6.wav
(1, 75)
8 /content/drive/MyDrive/3sec_audio_files/blues/blues.00000_chunk_7.wav
(1, 75)
9 /content/drive/MyDrive/3sec_audio_files/blues/blues.00000_chunk_8.wav
(1, 75)
10 /content/drive/MyDrive/3sec_audio_files/blues/blues.00000_chunk_9.wav


/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=286
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=286
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/pitch.py:153: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn("Trying to estimate tuning from empty frequency set.")
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=286
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=143
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=72
  n_fft, y.shape[-1]
/usr/loca

(1, 75)
11 /content/drive/MyDrive/3sec_audio_files/blues/blues.00000_chunk_10.wav
(1, 75)
12 /content/drive/MyDrive/3sec_audio_files/blues/blues.00001_chunk_0.wav
(1, 75)
13 /content/drive/MyDrive/3sec_audio_files/blues/blues.00001_chunk_1.wav
(1, 75)
14 /content/drive/MyDrive/3sec_audio_files/blues/blues.00001_chunk_2.wav
(1, 75)
15 /content/drive/MyDrive/3sec_audio_files/blues/blues.00001_chunk_3.wav
(1, 75)
16 /content/drive/MyDrive/3sec_audio_files/blues/blues.00001_chunk_4.wav
(1, 75)
17 /content/drive/MyDrive/3sec_audio_files/blues/blues.00001_chunk_5.wav
(1, 75)
18 /content/drive/MyDrive/3sec_audio_files/blues/blues.00001_chunk_6.wav
(1, 75)
19 /content/drive/MyDrive/3sec_audio_files/blues/blues.00001_chunk_7.wav
(1, 75)
20 /content/drive/MyDrive/3sec_audio_files/blues/blues.00001_chunk_8.wav
(1, 75)
21 /content/drive/MyDrive/3sec_audio_files/blues/blues.00001_chunk_9.wav
(1, 75)
22 /content/drive/MyDrive/3sec_audio_files/blues/blues.00001_chunk_10.wav
(1, 75)
23 /content/drive/

/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=176
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=176
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=88
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=44
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=22
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=11
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spec

(1, 75)
1551 /content/drive/MyDrive/3sec_audio_files/classical/classical.00043_chunk_10.wav
(1, 75)
1552 /content/drive/MyDrive/3sec_audio_files/classical/classical.00042_chunk_0.wav
(1, 75)
1553 /content/drive/MyDrive/3sec_audio_files/classical/classical.00042_chunk_1.wav
(1, 75)
1554 /content/drive/MyDrive/3sec_audio_files/classical/classical.00042_chunk_2.wav
(1, 75)
1555 /content/drive/MyDrive/3sec_audio_files/classical/classical.00042_chunk_3.wav
(1, 75)
1556 /content/drive/MyDrive/3sec_audio_files/classical/classical.00042_chunk_4.wav
(1, 75)
1557 /content/drive/MyDrive/3sec_audio_files/classical/classical.00042_chunk_5.wav
(1, 75)
1558 /content/drive/MyDrive/3sec_audio_files/classical/classical.00042_chunk_6.wav
(1, 75)
1559 /content/drive/MyDrive/3sec_audio_files/classical/classical.00042_chunk_7.wav
(1, 75)
1560 /content/drive/MyDrive/3sec_audio_files/classical/classical.00042_chunk_8.wav
(1, 75)
1561 /content/drive/MyDrive/3sec_audio_files/classical/classical.00042_chunk_9.wa

...

Streaming output truncated to the last 5000 lines.
(1, 75)
7360 /content/drive/MyDrive/3sec_audio_files/metal/metal.00085_chunk_9.wav
(1, 75)
7361 /content/drive/MyDrive/3sec_audio_files/metal/metal.00091_chunk_0.wav
(1, 75)
7362 /content/drive/MyDrive/3sec_audio_files/metal/metal.00091_chunk_1.wav
(1, 75)
7363 /content/drive/MyDrive/3sec_audio_files/metal/metal.00091_chunk_2.wav
(1, 75)
7364 /content/drive/MyDrive/3sec_audio_files/metal/metal.00091_chunk_3.wav
(1, 75)
7365 /content/drive/MyDrive/3sec_audio_files/metal/metal.00091_chunk_4.wav
(1, 75)
7366 /content/drive/MyDrive/3sec_audio_files/metal/metal.00091_chunk_5.wav
(1, 75)
7367 /content/drive/MyDrive/3sec_audio_files/metal/metal.00091_chunk_6.wav
(1, 75)
7368 /content/drive/MyDrive/3sec_audio_files/metal/metal.00091_chunk_7.wav
(1, 75)
7369 /content/drive/MyDrive/3sec_audio_files/metal/metal.00091_chunk_8.wav
(1, 75)
7370 /content/drive/MyDrive/3sec_audio_files/metal/metal.00091_chunk_9.wav
(1, 75)
7371 /content/drive/MyDrive/

/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=553
  n_fft, y.shape[-1]


(1, 75)
9860 /content/drive/MyDrive/3sec_audio_files/rock/rock.00032_chunk_10.wav
(1, 75)
9861 /content/drive/MyDrive/3sec_audio_files/rock/rock.00029_chunk_0.wav
(1, 75)
9862 /content/drive/MyDrive/3sec_audio_files/rock/rock.00029_chunk_1.wav
(1, 75)
9863 /content/drive/MyDrive/3sec_audio_files/rock/rock.00029_chunk_2.wav
(1, 75)
9864 /content/drive/MyDrive/3sec_audio_files/rock/rock.00029_chunk_3.wav
(1, 75)
9865 /content/drive/MyDrive/3sec_audio_files/rock/rock.00029_chunk_4.wav
(1, 75)
9866 /content/drive/MyDrive/3sec_audio_files/rock/rock.00029_chunk_5.wav
(1, 75)
9867 /content/drive/MyDrive/3sec_audio_files/rock/rock.00029_chunk_6.wav
(1, 75)
9868 /content/drive/MyDrive/3sec_audio_files/rock/rock.00029_chunk_7.wav
(1, 75)
9869 /content/drive/MyDrive/3sec_audio_files/rock/rock.00029_chunk_8.wav
(1, 75)
9870 /content/drive/MyDrive/3sec_audio_files/rock/rock.00029_chunk_9.wav
(1, 75)
9871 /content/drive/MyDrive/3sec_audio_files/rock/rock.00029_chunk_10.wav
(1, 75)
9872 /content/driv

/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=995
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=498
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=249
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=1024 is too small for input signal of length=125
  n_fft, y.shape[-1]


(1, 75)
9981 /content/drive/MyDrive/3sec_audio_files/rock/rock.00034_chunk_10.wav
(1, 75)
9982 /content/drive/MyDrive/3sec_audio_files/rock/rock.00040_chunk_0.wav
(1, 75)
9983 /content/drive/MyDrive/3sec_audio_files/rock/rock.00040_chunk_1.wav
(1, 75)
9984 /content/drive/MyDrive/3sec_audio_files/rock/rock.00040_chunk_2.wav
(1, 75)
9985 /content/drive/MyDrive/3sec_audio_files/rock/rock.00040_chunk_3.wav
(1, 75)
9986 /content/drive/MyDrive/3sec_audio_files/rock/rock.00040_chunk_4.wav
(1, 75)
9987 /content/drive/MyDrive/3sec_audio_files/rock/rock.00040_chunk_5.wav
(1, 75)
9988 /content/drive/MyDrive/3sec_audio_files/rock/rock.00040_chunk_6.wav
(1, 75)
9989 /content/drive/MyDrive/3sec_audio_files/rock/rock.00040_chunk_7.wav
(1, 75)
9990 /content/drive/MyDrive/3sec_audio_files/rock/rock.00040_chunk_8.wav
(1, 75)
9991 /content/drive/MyDrive/3sec_audio_files/rock/rock.00040_chunk_9.wav
(1, 75)
9992 /content/drive/MyDrive/3sec_audio_files/rock/rock.00040_chunk_10.wav
(1, 75)
9993 /content/driv

1100

In [ ]:
newly_engneered_df

,chroma_mean_0,chroma_mean_1,chroma_mean_2,chroma_mean_3,chroma_mean_4,chroma_mean_5,chroma_mean_6,chroma_mean_7,chroma_mean_8,chroma_mean_9,chroma_mean_10,chroma_mean_11,chroma_std_0,chroma_std_1,chroma_std_2,chroma_std_3,chroma_std_4,chroma_std_5,chroma_std_6,chroma_std_7,chroma_std_8,chroma_std_9,chroma_std_10,chroma_std_11,mfccs_mean_0,mfccs_mean_1,mfccs_mean_2,mfccs_mean_3,mfccs_mean_4,mfccs_mean_5,mfccs_mean_6,mfccs_mean_7,mfccs_mean_8,mfccs_mean_9,mfccs_mean_10,mfccs_mean_11,mfccs_mean_12,mfccs_std_0,mfccs_std_1,mfccs_std_2,mfccs_std_3,mfccs_std_4,mfccs_std_5,mfccs_std_6,mfccs_std_7,mfccs_std_8,mfccs_std_9,mfccs_std_10,mfccs_std_11,mfccs_std_12,cent_mean,cent_std,cent_skew,contrast_mean_0,contrast_mean_1,contrast_mean_2,contrast_mean_3,contrast_mean_4,contrast_mean_5,contrast_mean_6,contrast_std_0,contrast_std_1,contrast_std_2,contrast_std_3,contrast_std_4,contrast_std_5,contrast_std_6,rolloff_mean,rolloff_std,rolloff_skew,zrate_mean,zrate_std,zrate_skew,tempo,label
0,0.396309,0.341585,0.263590,0.100412,0.245896,0.259294,0.259644,0.246307,0.147863,0.210064,0.222643,0.286939,0.168174,0.140879,0.089389,0.082442,0.162034,0.174099,0.085476,0.091961,0.087232,0.114255,0.105992,0.160869,-194.805054,129.548203,-22.868769,42.500526,-0.256241,20.631479,-13.101333,16.572599,-12.250131,9.856705,-6.648044,8.342713,-1.780255,22.978418,11.370973,11.824155,9.550922,9.991274,5.946554,4.509854,6.157976,6.511330,8.567626,5.107795,5.267926,4.933665,1773.285877,410.176460,3.025547,22.825332,20.596504,22.891155,23.599423,21.765227,18.512826,39.091466,5.567421,4.816084,5.651457,5.124061,3.509231,2.024308,2.768790,3714.477539,1039.348387,1039.348387,0.067289,0.012274,0.041752,135.999178,blues
1,0.213842,0.161698,0.251429,0.145076,0.281398,0.363963,0.369807,0.448269,0.204334,0.105743,0.122278,0.203868,0.217887,0.104479,0.092536,0.047932,0.105395,0.100323,0.098354,0.121293,0.110252,0.108802,0.155059,0.149037,-202.909851,126.963280,-20.194355,53.591015,-8.311159,20.538086,-11.789995,20.042255,-10.255818,18.916189,-8.434890,7.709795,-6.451721,15.454389,10.274666,13.037852,9.732214,9.218394,5.520307,6.471244,5.976818,5.714737,4.797000,6.607670,5.773585,5.066738,1816.195860,301.169927,0.945127,22.436128,18.087297,21.900181,22.523867,22.509908,19.723049,38.558816,5.256100,4.077694,4.477236,3.884322,4.253553,2.970737,2.093325,3868.771221,819.646027,819.646027,0.073299,0.031132,0.889179,123.046875,blues
2,0.403105,0.306898,0.208723,0.082995,0.168394,0.218837,0.197697,0.305333,0.161147,0.241603,0.272439,0.331954,0.169150,0.137215,0.134439,0.086701,0.115263,0.056782,0.134531,0.109843,0.128096,0.182967,0.169463,0.131316,-206.807129,115.097427,-11.348315,50.981438,7.380768,23.697449,-7.132458,14.183143,-8.416517,7.921586,-2.288812,11.442759,0.044916,42.613430,11.408008,11.346096,11.409530,11.523343,6.504619,8.265283,6.190375,10.999379,4.733372,4.997000,6.308497,6.201508,1788.642783,333.650322,0.774920,23.706225,19.189425,23.025431,21.587071,22.423103,18.799707,39.185224,6.694738,4.961018,6.217921,4.261619,4.233219,2.311570,2.802231,3995.568660,892.103186,892.103186,0.057136,0.020458,0.392227,123.046875,blues
3,0.218068,0.176571,0.260356,0.122768,0.292104,0.377287,0.370132,0.336036,0.204973,0.281418,0.273202,0.171581,0.220974,0.066734,0.099626,0.061117,0.077734,0.077021,0.132787,0.085175,0.058740,0.072948,0.098389,0.115221,-192.478256,134.473557,-16.325451,41.555965,-5.911410,20.214130,-12.681191,16.396389,-9.523973,12.075930,-10.222750,7.592560,-0.273055,26.807074,11.373467,7.532714,9.861969,6.374863,8.364445,7.446799,6.216825,5.844546,4.470081,5.885951,5.294925,7.442510,1654.902168,335.136188,0.250880,23.088655,19.329863,21.807639,21.483658,22.247073,18.441560,38.689145,5.654919,4.438898,6.156516,4.213121,3.373180,1.866688,3.073826,3566.395358,962.611680,962.611680,0.061122,0.017311,0.553962,135.999178,blues
4,0.221119,0.181289,0.303179,0.084701,0.112070,0.236552,0.283621,0.451708,0.246292,0.185421,0.342043,0.277807,0.198743,0.178171,0.075098,0.071555,0.1

In [ ]:
def model_assess(model, title = "Default"):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    #print(confusion_matrix(y_test, preds))
    print('Accuracy', title, ':', round(accuracy_score(y_test, preds), 5), '\n')

In [ ]:
ndf = newly_engneered_df
label_encoder = preprocessing.LabelEncoder()
ndf['label'] = label_encoder.fit_transform(ndf['label'])

X = ndf.drop(['label'],axis=1)
y = ndf['label'] 

cols = X.columns
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)

# new data frame with the new scaled data. 
X = pd.DataFrame(np_scaled, columns = cols)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=111)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Naive Bayes
nb = GaussianNB()
model_assess(nb, "Naive Bayes")

# Stochastic Gradient Descent
sgd = SGDClassifier(max_iter=5000, random_state=0)
model_assess(sgd, "Stochastic Gradient Descent")

# KNN
knn = KNeighborsClassifier(n_neighbors=19)
model_assess(knn, "KNN")

# Decission trees
tree = DecisionTreeClassifier()
model_assess(tree, "Decission trees")

# Random Forest
rforest = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0)
model_assess(rforest, "Random Forest")

# Support Vector Machine
svm = SVC(decision_function_shape="ovo")
model_assess(svm, "Support Vector Machine")

# Logistic Regression
lg = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
model_assess(lg, "Logistic Regression")

Accuracy Naive Bayes : 0.46351 

Accuracy Stochastic Gradient Descent : 0.62789 

Accuracy KNN : 0.70586 

Accuracy Decission trees : 0.5864 

Accuracy Random Forest : 0.73706 

Accuracy Support Vector Machine : 0.74298 

Accuracy Logistic Regression : 0.65908 



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
